In [ ]:
from PIL import Image
from io import BytesIO
import json
import pandas as pd
import os

# Parquet 파일 로드
df = pd.read_parquet('./train-00000-of-00004-b4aaeceff1d90ecb.parquet')

# 이미지와 JSON 파일을 저장할 경로 설정
image_dir = '../data/medical_receipt/img'
json_dir = '../data/medical_receipt/ufo'

# 디렉터리 생성 (이미 존재하면 무시)
os.makedirs(image_dir, exist_ok=True)
os.makedirs(json_dir, exist_ok=True)

# 이미지 저장
for index, row in df.iterrows():
    image_data = row['image']['bytes']  # 이미지 바이너리 데이터 추출
    image = Image.open(BytesIO(image_data))  # 이미지 객체로 변환

    # 이미지 저장 (예: image_1.jpg, image_2.jpg ...)
    image_path = os.path.join(image_dir, f'image_{index + 1}.jpg')
    image.save(image_path)

# JSON 저장
for index, row in df.iterrows():
    ground_truth_str = row['ground_truth']  # ground_truth 열의 문자열 추출
    ground_truth_dict = json.loads(ground_truth_str)  # JSON 문자열을 딕셔너리로 변환

    # transcription 정보 추출
    transcriptions = [item['nm'] for item in ground_truth_dict['gt_parse']['menu']]

    # JSON 파일 저장 경로 설정 (예: image_1.json, image_2.json ...)
    json_path = os.path.join(json_dir, f'image_{index + 1}.json')
    ground_truth_dict['transcriptions'] = transcriptions  # 새롭게 transcription 리스트 추가

    with open(json_path, 'w', encoding='utf-8') as json_file:
        json.dump(ground_truth_dict, json_file, ensure_ascii=False, indent=4)

print("이미지와 JSON 파일 저장이 완료되었습니다.")


이미지와 JSON 파일 저장이 완료되었습니다.


In [2]:
# ground_truth 데이터 샘플 출력
for index, row in df.iterrows():
    ground_truth_str = row['ground_truth']
    ground_truth_dict = json.loads(ground_truth_str)
    print(json.dumps(ground_truth_dict, indent=4, ensure_ascii=False))  # 구조 확인을 위해 JSON 포맷으로 출력
    break  # 샘플 하나만 확인


{
    "gt_parse": {
        "menu": [
            {
                "nm": "Nasi Campur Bali",
                "cnt": "1 x",
                "price": "75,000"
            },
            {
                "nm": "Bbk Bengil Nasi",
                "cnt": "1 x",
                "price": "125,000"
            },
            {
                "nm": "MilkShake Starwb",
                "cnt": "1 x",
                "price": "37,000"
            },
            {
                "nm": "Ice Lemon Tea",
                "cnt": "1 x",
                "price": "24,000"
            },
            {
                "nm": "Nasi Ayam Dewata",
                "cnt": "1 x",
                "price": "70,000"
            },
            {
                "nm": "Free Ice Tea",
                "cnt": "3 x",
                "price": "0"
            },
            {
                "nm": "Organic Green Sa",
                "cnt": "1 x",
                "price": "65,000"
            },
            {
               

In [10]:
import json
import os

# JSON 파일이 있는 폴더 경로 (개별 JSON 파일)
json_folder = '../data/medical_receipt/ufo/'  # cord_json에서 ufo로 변경
output_path = '../data/medical_receipt/ufo/CORD_json.json'

# 기존 정보
info = {
    'year': 2024,
    'version': '1.0',
    'description': 'OCR Competition Data',
    'contributor': 'Naver Boostcamp',
    'url': 'https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000273/data/data.tar.gz',
}
licenses = {
    'id': '1',
    'name': 'For Naver Boostcamp Competition',
    'url': None
}
categories = [{'id': 1, 'name': 'word'}]

# COCO 데이터 초기화
img_id = 1
annotation_id = 1
images = []
annotations = []

# JSON 파일 이름 정렬
file_names = os.listdir(json_folder)
sorted_file_names = sorted(file_names, key=lambda x: int(x.split('_')[1].split('.')[0]))

# 파일 순회하며 COCO 포맷으로 변환
for file_name in sorted_file_names:
    if file_name.endswith('.json'):
        input_path = os.path.join(json_folder, file_name)

        with open(input_path, 'r') as f:
            file = json.load(f)

        # 이미지 정보 수집
        image = {
            'id': img_id,
            'width': file['meta']['image_size']['width'],
            'height': file['meta']['image_size']['height'],
            'file_name': f'image_{img_id}.jpg',  # 이미지 파일명 매칭
            "license": 1,
            "flickr_url": None,
            "coco_url": None,
            'data_captured': None
        }
        images.append(image)

        # 어노테이션 정보 수집
        for ann_info in file['valid_line']:
            for word_info in ann_info['words']:
                quad_info = word_info['quad']
                x1 = quad_info['x1']
                y1 = quad_info['y1']
                x2 = quad_info['x2']
                y3 = quad_info['y3']

                # bbox와 segmentation 계산
                min_x = x1
                min_y = y1
                width = x2 - x1
                height = y3 - y1

                segmentation = [
                    [min_x, min_y, min_x + width, min_y, min_x + width, min_y + height, min_x, min_y + height]
                ]

                coco_annotation = {
                    "id": annotation_id,
                    "image_id": img_id,
                    "category_id": 1,
                    "segmentation": segmentation,
                    "area": width * height,
                    "bbox": [min_x, min_y, width, height],
                    "iscrowd": 0,
                    'tags': ['Auto']
                }
                annotations.append(coco_annotation)
                annotation_id += 1

        img_id += 1

# COCO 포맷 데이터 합치기
coco = {
    'info': info,
    'images': images,
    'annotations': annotations,
    'licenses': licenses,
    'categories': categories
}

# 최종 COCO JSON 파일로 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(coco, f, indent=4, ensure_ascii=False)

print("COCO JSON 파일이 성공적으로 저장되었습니다.")


COCO JSON 파일이 성공적으로 저장되었습니다.


In [11]:
from typing import Dict
import json
import datetime

# 현재 시간 기록
now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 입력 및 출력 경로 설정
input_path = '/data/ephemeral/home/level2-cv-datacentric-cv-24/data/medical_receipt/ufo/CORD_json.json'
output_path = '/data/ephemeral/home/level2-cv-datacentric-cv-24/data/medical_receipt/ufo/_CORD_json.json'

# UFO 포맷 초기화
ufo = {
    'images': {}
}


In [12]:
def coco_bbox_to_ufo(bbox):
    min_x, min_y, width, height = bbox
    return [
        [min_x, min_y],
        [min_x + width, min_y],
        [min_x + width, min_y + height],
        [min_x, min_y + height]
    ]

def coco_to_ufo(file: Dict, output_path: str) -> None:
    anno_id = 1
    for annotation in file['annotations']:
        file_info = file['images'][int(annotation['image_id'])-1]
        image_name = file_info['file_name']
        if image_name not in ufo['images']:
            anno_id = 1
            ufo['images'][image_name] = {
                "paragraphs": {},
                "words": {},
                "chars": {},
                "img_w": file_info["width"],
                "img_h": file_info["height"],
                "tags": ["autoannotated"],
                "relations": {},
                "annotation_log": {
                    "worker": "",
                    "timestamp": now,
                    "tool_version": "LabelMe or CVAT",
                    "source": None
                    },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                    }
                }
            
            # anno_id = 1
        ufo['images'][image_name]['words'][str(anno_id).zfill(4)] = {
            "transcription": "",
            "points":  coco_bbox_to_ufo(annotation["bbox"]),
            "orientation": "Horizontal",
            "language": None,
            "tags": ['Auto'],
            "confidence": None,
            "illegibility": False
        }
        anno_id += 1

    with open(output_path, "w") as f:
        json.dump(ufo, f, indent=4)

In [13]:
# COCO JSON 파일 로드 및 변환 수행
with open(input_path, 'r', encoding='utf-8') as f:
    file = json.load(f)

# COCO -> UFO 변환
coco_to_ufo(file,output_path)

# 변환된 UFO 포맷 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(ufo, f, indent=4, ensure_ascii=False)

print("COCO 포맷을 UFO 포맷으로 변환하여 저장했습니다.")

COCO 포맷을 UFO 포맷으로 변환하여 저장했습니다.


In [ ]:
import os
import json
import datetime

# 현재 시간 기록
now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 입력 및 출력 경로 설정 (필요한 경우 수정)
box_folder = '/path/to/box/files'  # box 파일들이 있는 폴더 경로
img_folder = '/path/to/img/files'  # img 폴더에 해당 이미지가 있다고 가정
output_path = '/path/to/ufo_output.json'

# UFO 포맷 초기화
ufo = {
    'images': {}
}

def parse_box_line(line):
    # 각 줄의 데이터 파싱
    parts = line.strip().split(',')
    if len(parts) < 5:
        return None, None  # 최소 좌표 4개 + 텍스트가 필요함
    coordinates = list(map(int, parts[:8]))  # 좌표 부분
    text = ','.join(parts[8:]).strip()  # 좌표 이후의 모든 내용이 텍스트로 간주
    points = [
        [coordinates[0], coordinates[1]],
        [coordinates[2], coordinates[3]],
        [coordinates[4], coordinates[5]],
        [coordinates[6], coordinates[7]]
    ]
    return points, text

# box 파일 처리
for box_file in os.listdir(box_folder):
    if box_file.endswith('.txt'):
        img_id = box_file.split('.')[0]  # 파일명에서 이미지 ID 추출
        img_path = os.path.join(img_folder, f"{img_id}.jpg")

        # 이미지에 대한 UFO 구조 생성
        ufo['images'][img_id] = {
            "paragraphs": {},
            "words": {},
            "chars": {},
            "img_w": None,  # 나중에 실제 이미지 크기를 넣을 수 있음
            "img_h": None,  # 나중에 실제 이미지 크기를 넣을 수 있음
            "tags": ["autoannotated"],
            "relations": {},
            "annotation_log": {
                "worker": "",
                "timestamp": now,
                "tool_version": "Auto-converted",
                "source": None
            },
            "license_tag": {
                "usability": True,
                "public": False,
                "commercial": True,
                "type": None,
                "holder": "Upstage"
            }
        }

        # 각 box 파일을 읽어 단어 정보 저장
        with open(os.path.join(box_folder, box_file), 'r', encoding='utf-8') as f:
            for anno_id, line in enumerate(f, 1):
                points, text = parse_box_line(line)
                if points is None:
                    continue  # 잘못된 형식의 라인은 무시

                # UFO 형식으로 단어 추가
                ufo['images'][img_id]['words'][str(anno_id).zfill(4)] = {
                    "transcription": text,
                    "points": points,
                    "orientation": "Horizontal",
                    "language": None,
                    "tags": ['Auto'],
                    "confidence": None,
                    "illegibility": False
                }

# 변환된 UFO 포맷 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(ufo, f, indent=4, ensure_ascii=False)

print("Box 파일을 UFO 포맷으로 변환하여 저장했습니다.")